In [79]:
import pandas as pd
import numpy as np

from Utils.Utils import GridSearch as Gs
from pandas import DataFrame

In [80]:
LGBMRanker = Gs.load_model(name="../BlackBox/lmart/saved_models/LGBMRanker_unbalanced")

In [81]:
FIGSReg = Gs.load_model(name="../GlassBox/figs/saved_models/FIGSReg_unbalanced")
FIGSClass = Gs.load_model(name="../GlassBox/figs/saved_models/FIGSClass_unbalanced")

In [82]:
EBMReg = Gs.load_model(name="../GlassBox/ebm/saved_models/EBMReg_unbalanced")
EBMClass = Gs.load_model(name="../GlassBox/ebm/saved_models/EBMClass_unbalanced")

In [83]:
def predict_rank(model, X:DataFrame, top_k:int=15):
    y_pred = model.predict(np.asarray(X.iloc[:,2:13].values))
    y_pred = pd.DataFrame(y_pred, index=X.index, columns=["lambdas"])
    dt_final = pd.merge(X, y_pred, left_index=True, right_index=True)
    return dt_final.sort_values("lambdas",ascending=False)[["kId","labels"]].head(top_k)

In [84]:
test_set = pd.read_csv("../outputs/scores/scores_ts.csv")

In [85]:
test_set[test_set["labels"]==4]

,qId,kId,Education,City,Skills,SoftSkills,Age,Language,Certificates,Experience,Salary,SmartWork,Experience_abroad,score,w_score,labels
487,5,236,0.6,0.443680,1.00,1.000000,0.857143,0.000000,0.0,1.000,1.0,0,0,5.788108,0.838096,4
2500,48,151,1.0,0.742906,1.00,0.666667,1.000000,0.000000,1.0,0.875,1.0,0,0,7.213575,0.862468,4
3169,43,198,1.0,0.459157,0.75,0.400000,0.000000,0.666667,0.0,1.000,1.0,0,1,6.279169,0.829507,4
3672,35,236,1.0,0.796217,1.00,1.000000,0.714286,1.000000,0.0,1.000,0.0,1,0,7.439241,0.846146,4


In [95]:
job = test_set[test_set["qId"]==15]

# Perfect rank

In [87]:
job[["kId","labels"]].sort_values("labels",ascending=False).head(15)

,kId,labels
2839,228,3
171,256,3
4808,117,3
2377,67,2
2712,194,2
2635,34,2
2599,85,2
2561,150,2
4055,167,2
4095,226,2


# LGBMRanker

In [88]:
predict_rank(LGBMRanker, job)

,kId,labels
171,256,3
2839,228,3
4808,117,3
2712,194,2
902,265,1
2259,281,2
1729,133,1
2307,151,2
3817,206,1
948,77,1


# FIGSRegressor

In [89]:
predict_rank(FIGSReg, job)

,kId,labels
171,256,3
2839,228,3
4808,117,3
2307,151,2
2635,34,2
902,265,1
1729,133,1
3816,61,2
1453,230,2
3321,264,1


# EBM Regressor

In [90]:
predict_rank(EBMReg, job)

,kId,labels
171,256,3
2839,228,3
4808,117,3
2307,151,2
902,265,1
2635,34,2
1729,133,1
3816,61,2
2712,194,2
2259,281,2


# FIGS Classifier

In [91]:
predict_rank(FIGSClass, job)

,kId,labels
91,52,1
3583,101,2
3839,17,1
3817,206,1
3816,61,2
3693,93,2
3691,24,1
3684,242,1
3649,222,1
3648,183,2


# EBM Classifier

In [92]:
predict_rank(EBMClass, job)

,kId,labels
171,256,3
948,77,1
2712,194,2
3816,61,2
1453,230,2
2259,281,2
2307,151,2
3583,101,2
902,265,1
2635,34,2
